#### Import libraries:

In [1]:
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from numpy import expand_dims
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
import random
import requests
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array 
import zipfile


# ModelGenerator2

In deze tweede 'model generator' genaamd **'modelGenerator2'** wil ik de data op dezelfde manier inladen als wij geleerd hebben in de Notebook Numpy2. De reden waarom ik een tweede model generator maak is omdat ik controlle wil hebben over de data die ik het model aanbied. In het bijzonder over de aantallen binnen de subklasses van de train folder. Ik wil kijken wat het effect is op het model wanneer ik een gelijkmatig verdeelde trainingset aanbied. 

Dit resulteerd in: 
470 afbeeldingen voor 'Blotch_apples'
470 afbeeldingen voor 'Normal_apples'
470 afbeeldingen voor 'Rot_apples'
470 afbeeldingen voor 'Scab_apples'


Het is met "tf.keras.utils.image_dataset_from_directory" niet gelukt om deze gelijkmatigheid te bereiken. VERVOLG


## Inladen van de data

De 'augumented' data is onderscheiden door de soort bewerking voor de bestandsnaam toe te voegen. Deze zijn te herkennnen aan de namen 'flipped', 'black' en 'duotone'.

In [2]:
imgHeight = 100
imgWidth = 100

In [3]:
txtFiles = list() # deze lijst wordt aangemaakt om later hier onder te kunnen gebruiken bij het aanmaken van de lijsten met bestandsnamen uit de verschillende klasses.

In [4]:
categories = ['Blotch', 'Normal', 'Rot', 'Scab']

trainDir = "data/Train_cleaned_and_augumented"

for category in categories:
  imgDir = f"{trainDir}/{category}_Apple/"
  for filename in os.listdir(imgDir): 
    txtFile = os.path.join(imgDir, filename)
    txtFiles.append(txtFile) 
    # print(txtFile)


In [5]:
type(txtFiles)

list

## Data inlezen

In [6]:
txtFiles = [x for x in txtFiles if ".jpeg" in x] #  jpeb filteren.
print(len(txtFiles))

2949


In [7]:
imageObjects= np.zeros([len(txtFiles), imgHeight, imgWidth, 3]) 

i = 0

for txtFile in txtFiles:
    imageObjects[i] = np.asarray(Image.open(txtFile)).astype('uint8')/255 # Afbeelding wordt genormaliseerd.
    i += 1

imageObjects = np.array(imageObjects) # List wordt omgezet in array.


ValueError: could not broadcast input array from shape (215,234,3) into shape (100,100,3)

In [8]:
np.shape(imageObjects)

(2949, 100, 100, 3)

### Gebruik de bestandsnamen voor het verkrijgen van de labels

In [ ]:
# imageLabels = np.empty(len(txtFiles), dtype = 'S20')

i = 0

for label in txtFiles:
    txtFiles[i] = label.split('/')[2]
    i += 1
    
labelNames, labelNumbers = np.unique(txtFiles, return_inverse=True)
labelDict = dict(zip(np.unique(labelNumbers), labelNames))

print(labelNames)
print(labelNumbers)

np.array(np.unique(labelNumbers, return_counts=True)).T # demonstratie van methode unique. 

In [ ]:
txtFilesSel = list()
imageObjectsSel = list()
indexRanges = [(0,888),(888,1360),(1360,2272),(2272,2949)] # kijk of je dit nog kunt automatiseren door te koppelen aan de lengte van...

for indexR in indexRanges:
    for number in range (470): # Nadeel -> wordt elke keer opnieuw bepaald
        randomIndex = random.choice(range(indexR[0],indexR[1]))
        txtFilesSel.append(txtFiles[randomIndex])
        imageObjectsSel.append(imageObjects[randomIndex])

imageObjects = np.array(imageObjectsSel) # omzetten van lijst naar array


## Data splitten

In [ ]:
np.shape(imageObjects)

In [ ]:
trainSet, testSet, trainLabels, testLabels = train_test_split(imageObjects, labelNumbers, stratify = labelNumbers, train_size = 0.75, random_state=42)
print(trainSet.shape)
nrOfImages = len(trainSet)

In [ ]:
print(nrOfImages)

## Data inspecteren

In [ ]:
def inspectData():
    
    plt.figure(figsize=(10,10))
    
    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        number = i
        plt.imshow(trainSet[number])
        plt.xlabel(labelNames[trainLabels[number]]) 
    
    plt.show()

In [ ]:
inspectData()

## Bouwen van het model

In [ ]:
# #model 2 geinspireerd van nick nochnack

model = Sequential()
num_classes = len(categories)

model.add(layers.Rescaling(1./255, input_shape=(imgHeight, imgWidth, 3)))
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(layers.Dense(num_classes))

model.summary()

### Definieer de trainparameters 

In [ ]:
lossFunction = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gdAlgorithm = keras.optimizers.Adam(learning_rate=0.001)
nrOfEpochs = 5

## Train het model

In [ ]:
model.compile(optimizer=gdAlgorithm, loss=lossFunction, metrics="accuracy")


In [ ]:
history = model.fit(trainSet, trainLabels, epochs=nrOfEpochs, batch_size=64, verbose=2)

Ga gericht opzoek naar de omzetting van numpy naar tensor 

In [ ]:
# model.save(os.path.join(f'models/[new]px.h5'))

## Evalueer het model

In [ ]:
# test_data_dir = '/Users/boyfrankclaesen/workspace/makeAIWork2/projects/apple_disease_classification/classifier/data/Test'

# test_ds = tf.keras.utils.image_dataset_from_directory(
#   test_data_dir,
#   seed=123,
#   image_size=(imgHeight, imgWidth),
#   batch_size=batch_size)

In [ ]:
# scores = model.evaluate(test_ds)